# Inference with Models

An X-ray transient monitor has detected a new source, also observed by LIGO. This is exciting! However, we don't know what it is, so we'd like to know what Fermi observed. For the purpose of this toy problem, we are going to ignore the question of whether Fermi observed the source at all or not (although that's an important problem, too!). For the purpose of this tutorial, we'll assume we know that Fermi observed something significant! Hooray!

What we'd like to know: what are the details of the source that Fermi observed? We'd like to do a multi-wavelength Spectral Energy Distribution, so knowing the total flux that Fermi observed would be very useful to help piece that together. We'd also like to know the precise position of the source. 

The purpose of the following analysis is to infer the physical properties (its flux and its position) so that we can use these to help us understand the nature of the new, exciting source we've observed. 

**Important**: This tutorial is based on a toy problem, and while it pretends to be using Fermi, it's not really using Fermi. Any Fermi-related things are entirely made up, and should not be used for actual analysis! This is a toy problem designed to highlight the statistical concepts, and consequently strips out a whole range of important calibration issues for the sake of simplicity and clarity (and also, I don't actually know that much about the Fermi/LAT). 

## Imports

Let's start by importing some libraries that will definitely come in handy!

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# I like using the plotting library seaborn to set style defaults
# If you don't have seaborn or don't want to use it, comment out the 
# two lines below
import seaborn as snsl
sns.set_style('white')

import numpy as np
import scipy.stats

There are three text files with image data in the folder. 


**Exercise**

Load the one that corresponds to your group number (you can use `numpy.loadtxt` or `pandas.read_csv` or whatever else you'd like):

The data file contains an image, which is 300 by 300 pixels wide. Let's take a look:

In [ ]:
plt.figure()
plt.imshow(counts)

We can generate x and y coordinates for these using `numpy.meshgrid`:

In [ ]:
img_size = 300
x, y = np.meshgrid(np.linspace(0,img_size, num=img_size),
                   np.linspace(0,img_size,num=img_size))

### Writing Down a Model

First, we're going to need some kind of model to represent that data! Usually, we need something like a Point Spread Function (PSF) that describes how photons get distributed onto bins (or pixels, or whatever your detector has).

Let's assume that the Fermi documentation tells us that the Fermi PSF is approximately Gaussian (I don't know that it is, and I kind of doubt it, so make sure to check for applications that are not this toy problem!). I've written down a 2D Gaussian for you below for your convenience. You might want to check that it doesn't include any bugs, though. :)

In [ ]:
def gauss_psf(x, y, amp, mean1, mean2, std, bkg):
    """
    Gaussian Point Spread Function
    
    Parameters
    ----------
    x, y: (300,300)-size arrays for the x and y coordinates
    
    amp: float
        The amplitude of the Gaussian. Because the Gaussian *integrates* to 1, 
        this corresponds to the total flux
        
    mean1, mean2 : float, float
        The x- and y-positions of the peak of the Gaussian
    
    std : float
        The width (standard deviation) of the Gaussian
        
    bkg : float
        A constant describing the background noise level.
    
    Returns
    -------
    gpsf : (300,300) array
        An array with the model flux values as x and y positions. 
    
    
    """
    norm_term = amp / (2. * np.pi * std**2.)
    exp_term = np.exp(-((x - mean1)**2. / (2. * std**2.) + (y - mean2)**2. / (2. * std**2.)))
    gpsf = norm_term * exp_term + bkg
    return gpsf

**Exercise**

Guess some reasonable parameters that might allow you to generate model fluxes similar to those in the data you loaded. Then plot both the data and the model fluxes next to each other and compare. How close did you get with the parameters you guessed?

Hint: if it's too hard to compare the two images, you can also take a vertical or horizontal slice out of the image and plot the data and model for those:

So now we have a model with some parameters, and some data. How do we bring those two together? 

## Writing Down a Likelihood

As we've learned in the lecture, this is what a likelihood is for! The likelihood describes how to relate the process we believe generated the emission (here, the flux produced by the source) and the process that modified the emission before it was recorded in the telescope (here, the point spread function) connect to the data recording process (here, we recorded individual photons).

**Exercise**: Can you guess from the data (and from what you know about Fermi data), what kind of likelihood might be appropriate here? Discuss with your group (solution below).

**Solution**

You think this data probably ought to be modelled with a Poisson distribution, but a collaborator tells you that the Poisson distribution is expensive to compute, and it probably won't make much of a difference, so just use a Gaussian likelihood instead.

The Gaussian likelihood is defined based on the normal distribution (unsurprisingly), so that the observed value $c$ in each pixel is picked from the following distribution:

$$
P(c | \mu, \sigma) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp\left(\frac{(c - \mu)^2}{2\sigma^2}\right)
$$

where $\mu$ is the expected flux and $\sigma$ the uncertainty. For the data we have, you can assume that $\sigma \approx \sqrt{c}$.

The function we are interested in is the *likelihood* of the data. The likelihood describes the probability of observing the data we've measured, conditioned on a *physical* model (your surface brightness model) and a *statistical* model (the Gaussian distribution). The physical model describes what you expect your image to look like if there was no noise. The statistical model describes the uncertainty in the measurements because there is noise present. 
The likelihood function is the product of the probabilities of all pixels, which is the probability of observing a number of counts in a pixel, $c_i$ under the assumption of a model count rate in that same pixel, $\mu_i$, multiplied together for all $N$ pixels.

All in all, the Gaussian likelihood for a given physical model $m(\mathbf{\theta})$, which depends on a set of $K$ parameters $\mathbf{\theta} = \{\theta_1, \theta_2, ... , \theta_k\}$ looks like this:

$$L(\mathbf{\theta}) = P(\mathbf{c}|\mathbf{\theta}, H) = \prod_{i=0}^N{\frac{1}{\sqrt{2\pi \sigma^2}} \exp\left(-\frac{(c - \mu)^2}{2\sigma^2}\right)}$$

In practical terms, the likelihood is often very large or very small, and it is more convenient to work with the logarithm of the likelihood:

$$ 
\log{(L(\mathbf{\theta}))} = \sum_{i=0}^{N}{-\frac{1}{2}\log{(2\pi\sigma^2)} - \frac{(c - \mu)^2}{2\sigma^2}}
$$


**Exercise**: Complete the likelihood function below:

In [1]:
logmin = 1000000000000.

def loglikelihood(pars, x, y, counts):
    """
    A Gaussian log-likelihood.
    
    Parameters
    ----------
    pars : iterable
        A list or array of parameters
        
    x, y: (300,300) array
        arrays for the x- and y-coordinate
    
    counts : (300,300) array
        An array with count values
    
    """

    ## call the model function (i.e. the gaussian PSF), use parameter set pars and
    ## the coordinate grids to compute the model image

    
    ## compute Gaussian likelihood

    
    
    ## deal with NaN and inf values of the log-likelihood
    ## if the likelihood is either NaN or inf, choose a ridiculously small value
    ## to make the model *really* unlikely, but not infinite
    if np.isnan(loglike):
        loglike = -logmin
    elif loglike == np.inf:
        loglike = -logmin

    return loglike

Write down the parameters you guessed above to generate your example model fluxes, and then calculate the log-likelihood of these parameters:

In [ ]:
test_pars = # add parameters here

In [ ]:
loglikelihood(test_pars, x, y, counts)

## Estimating the "Best" Parameters

The likelihood gives us an estimate for the probability of having recorded our observing data given some underlying "true" model and model parameters. However, we don't know what these true parameters are; that's exactly what we're trying to estimate! We can do so by stepping through different parameter values, and recording the likelihood of having observed the data, given the assumption that these parameters are true. We then pick the parameters that give us the highest likelihood value as the "best" parameters given our observed data. This is called **maximum likelihood estimation**. 

It is a somewhat subtle, but important distinction that the likelihood is *not* a probability distribution of the parameters, but of the data. It assumes that we are drawing random realizations from an underlying model, given a model and parameters that are known to be true. While you can make meaningful statistical inferences about a scientific problem this way, there are some cases where the structure of the likelihood as a probability of the data makes your life a bit harder. Later, we will talk about how Bayes' theorem gives you a convenient way to flip the probabilities around, and get a probability distribution of the parameters given an observed data set. 
[This paper]() by David W. Hogg gives a great intuitive introduction to the topic of why this matters.


### Optimization

So how do we search for the maximum likelihood in practice? One thing you can do is make a grid of parameters, calculate the likelihood for each one, and then simply pick the largest one. This can work great, but also has some significant drawbacks: for example, this might work great when you have only one or two parameters to estimate. Once you have more than that, however, this becomes very unwieldy and computationally expensive, because you have to generate a grid over as many dimensions as you have parameters. Secondly, what should the resolution of your grid be? What if you have a likelihood that's zero almost everywhere except for a small region of parameter space? If you make your grid too coarse, you run the risk of never actually getting close to the maximum likelihood at all. If you make it fine-grained, you are wasting a lot of computational time on regions of parameter space that are entirely uninteresting. 

One common way is to use **optimization**. Optimization algorithms usually start at a single point in parameter space, and then try to find intelligent ways to pick the next step where to estimate the likelihood. Often, this involves calculating the *gradient* of the likelihood to inform the algorithm where to step next. There's a wide range of different algorithms out there, suited to a number of different problems, but one good place to start is the `scipy.optimize` sub-module. 

One subtlety about optimization is that while we are trying to find the parameters where the likelihood has its *maximum*, most optimization algorithms are in practice *minimization* routines. That is, they attempt to find the minimum of a function. 

In practice, this is easily dealt with by defining the inverse of the log-likelihood, and then using this in our minimization algorithm:

In [ ]:
negloglike = lambda pars, x, y, poisson_flux: -loglikelihood(pars, x, y, poisson_flux)

One crucial component of optimization is picking good starting parameters. Bad starting parameters can get you stuck in local optima, that can be far away from the true optimum. The problem is that you don't know what the true optimum is, so you can't really know whether you got stuck in a local optimum. One thing you can do is start the optimization multiple times from different starting parameters, and then pick the outcome that has the highest value for the maximum likelihood. However, for some problems it's also possible to pick reasonably good starting parameters from the data itself.

**Exercise**: Can you think of ideas for how to pick starting parameters for the optimization? 

Let's pull all of these together into a single set of parameters:

In [ ]:
start_pars = # add your starting parameters here

print(start_pars)

Okay, now we've got all of our starting parameters. 

Let's run `scipy.optimize.minimze` to see what we get:

In [ ]:
res = scipy.optimize.minimize(negloglike, start_pars, args=(x, y, counts), method='bfgs', tol=1)

The output of `scipy.optimize.minimize` is an object with a wealth of useful information. 

Here are some important ones.

You'll want to pay attention to the attributes `success` and `message`, which give you information about whether the algorithm thinks it's succeeded to find the minimum of the function, and gives you some information (though it's usually pretty arcane), why it did or did not:

In [ ]:
print(res.success)
print(res.message)

In our case, the algorithm terminated successfully. Hooray!
When this is not the case, the error messages can usually be pretty arcane and hard to debug. It's then worth hunting around the internet for help on what they mean and what the issues might be.

The `fun` attribute gives you the function value at termination. This is the maximum likelihood we're looking for (or at least its inverse:

In [ ]:
print("Maximum likelihood: " + str(-res.fun))

The other useful information you'll want is stored in the `x` attribute, which has the parameters at which the optimizer has found the maximum likelihood:

In [ ]:
print("Best-fit parameters: " + str(res.x))

**Exercise**: Compute a model image for the best-fit model parameters, then compare the model to your data. You can compare the images again side by side, or you can plot slices through the image and model directly in one plot (the latter is probably more useful in comparing them):

**Exercise at home**: `scipy.optimize.minimize` has a number of available algorithms. Take a look at the documentation and explore 2-3 algorithms. Do all of them give you the same output? Do you get noticeably different parameters? Do you think they are better or worse than the standard BFGS algorithm we've chosen above?

### Parameter Uncertainties

We'd also like to know the uncertainties on our parameters. One estimate you can gain from the optimization process is to use the *inverse Hessian*. Many algorithms use the first and second derivatives of the likelihood as information on where to step next. We can use these estimates to calculate a measure of the width of the likelihood, which in turn can be interpreted as uncertainties on the parameters. 

**Note**: Not all minimization algorithms return the *inverse Hessian*, which you can interpret as a covariance matrix. If you are using an algorithm that doesn't, you can still often calculate a numerical approximation (there's a routine e.g. in the `statsmodels` package). 

**Important**: The calculation of uncertainties this way makes strong assumptions about the shape of the likelihood, which may or may not be appropriate for your problem. In particular, it makes the assumption that the likelihood is approximately Gaussian in shape (and so your uncertainties are symmetric). Some problems, especially those with complex physical models, may have more complicated shapes for the likelihood, or a likelihood that is asymmetric. In this case, the uncertainties derived in this way are not reliable. 

In [ ]:
print("Parameter uncertainty estimates: " + str(np.sqrt(np.diag(res.hess_inv))))

## Priors

Prior probability distributions describe the information we have about our problem and parameters before we've had a chance to look at the data. They usually describe probability distributions over parameters that encode information from the literature, or just known physics (for example, when fitting the optical spectrum of a sun-like star, we can automatically exclude certain temperatures from the prior, below which hydrogen would not fuse to helium, and which therefore are not physically meaningful temperatures). 

Let's assume that the initial transient detection came from the LIGO gravitational wave observatory. This is exciting, because electromagnetic counterparts to gravitational wave sources are scientifically very interesting. The LIGO observatory can't locate sources very well: their detection maps often stretch across a big stripe of the sky. However, they do give us *some* information about where the source we are interested in should be located on the sky.

For the purpose of this toy exercise, we're going to assume the following position estimates from LIGO:
x = 150 +/- 20
y = 180 +/- 100

We also have some information about the background and the Fermi PSF. For example, we know that the width of the Fermi PSF is about 12 +/- 3 pixels, and the background in this part of the sky is about 0.5 counts/pixel.

However, because our information on this source comes from LIGO, we don't really know anything about the brightness (i.e. amplitude) of the source in gamma-rays.

**Exercise**: Can we incorporate this information as a prior? What kind of shape would you choose for your prior? (Solution below).


**Hint**: `scipy.stats` has a wealth of distributions that are useful here. For example, to define a Gaussian prior with a mean of 2 and a variance of 1, we can use `scipy.stats.norm(2,1).logpdf(x)` to calculate the log-probability of value `x` given that normal prior.

**Solution**: 

This is what's called an **informative prior**. Informative priors are those where we have meaningful information prior to looking at the data to help us constrain the problem. Note that priors, in general 

Let's assume that LIGO told us their positional probabilities are approximately Gaussian, so we can pick a Gaussian prior for the positional parameters:

In [ ]:
pos1_prior = scipy.stats.norm(150, 20).logpdf
pos2_prior = scipy.stats.norm(180, 100).logpdf

We unfortunately don't know much about the amplitude in advance. For this parameter, we'll need what's called an **uninformative prior**. These are priors that encode our lack of information. Naively, you might expect that an uninformative prior would assign the same probability to each possible value across a range. However, for some parameters, this turns out not to be a good, uninformative choice. This is especially true for parameters that can range over multiple orders of magnitude. The problem here is that when you assign the same probability to all values, then the probability of a value between 0.1 and 0.2 is *much* smaller than the probability of a value between 1 and 2. To avoid this, we use what's called a *Jeffrey's prior*, which assigns equal probability to the *logarithm of the parameter.

In [ ]:
np.log(30000)

In [ ]:
logamp_prior = scipy.stats.uniform(-2, 15).logpdf

Finally, we also need a prior for the width. For this parameter, let's assume we know from the Fermi documentation that the PSF is about 12 +/- 3 pixels. Again, we'll translate this to a Gaussian prior.

The prior for the background parameter is also Gaussian, and based on our (presumed) knowledge of the gamma-ray sky. For this, we have a prior estimate of 0.5 +/- 0.1.

**Exercise**

Write down the log-priors for the width and the background:

Note that in principle, we could construct more complicated priors. For example, imagine that we know that the width of the PSF depends on the incoming flux (so on our amplitude parameter). We could construct a prior where the width prior actually depends on the amplitude parameter. 

However, today we'll forge ahead with the priors we have. According to Bayes' rule, the prior probability density for all parameters is the product of the individual densities. Because we've designed them as logarithms above, this then becomes a sum.

**Exercise**

Complete the function below to construct the prior of the whole set of parameters.
Make sure to account for the fact that the amplitude prior is defined on the *logarithm* of the amplitude parameter, not on the parameter itself.

In [ ]:
def logprior(pars):
    amp = pars[0]
    pos1 = pars[1]
    pos2 = pars[2]
    width = pars[3]
    bkg = pars[4]
    
    lp = pos1_prior(pos1) + # complete this function
    
    return lp

Let's compute the prior for the starting parameters you defined earlier:

In [ ]:
logprior(start_pars)

## The Posterior

We are now set up to calculate the posterior probability density. The posterior density is essentially the product of the likelihood and prior (except for a normalization factor, which is important in the context of model comparison, but which we can safely neglect for parameter estimation). 

Again, defining our quantities in terms of their logarithm means that we can add them together:

In [ ]:
def logposterior(pars, x, y, poisson_flux):
    lpost = loglikelihood(pars, x, y, poisson_flux) + logprior(pars)
    
    if np.isnan(lpost):
        lpost = -logmin
    elif lpost == np.inf:
        lpost = -logmin
        
    return lpost

In [ ]:
logposterior(test_pars, x, y, counts)

In [ ]:
logposterior(start_pars, x, y, counts)

## Estimating the Posterior

In only very rare special cases, the posterior probability density is analytically tractable, that is, you can write down a functional form for it. This is particularly the case if you have defined what are called *conjugate* priors, which are priors that specifically make the posterior analytically tractable. Being able to cast your priors into a conjugate form is always an advantage, but rare in practice, especially when your problem has many parameters. You can find conjugate priors for commong likelihoods in various forms online. 

Alternatively, you can do optimization on the posterior the same way that we've done on the likelihood. This is called *maximum a posteriori estimation* and allows you to get quick parameter estimates that take our prior knowledge into account.

**Exercise**

Write down the code to do a maximum-a-posteriori estimation in the same way we've done above with the maximum likelihood estimation, using `scipy.optimize.minimize`. 


**Optional Exercise**: Start from different positions, and see how this changes the result. What happens if you pick starting values outside of the support of the prior (e.g. out of the bounds of flat priors, or very far out in the wings of a Gaussian prior)?

## Markov Chain Monte Carlo

However, there's a different way to characterize the posterior probability density. One advantage of writing down a posterior is that it gives us probability distributions over *parameters*, which we're interested in, especially if these distributions are not as well-behaved as required by typical optimization to give meaningful estimates.

Again, we could generate a grid and then plot the results. However, the same problems surrounding how to pick the grid size and computational inefficiencies still apply. 

Instead, what we want is an algorithm that explores parameter space relatively efficiently, but is also not concerned *just* with finding the maximum value, but rather can give us a fuller description of the probability distribution. 

Enter Markov Chain Monte Carlo! A full tutorial on MCMC is beyond the scope of this set of exercises, so if you're interested in more details, take a look at [this paper](https://iopscience.iop.org/article/10.3847/1538-4365/aab76e/meta). For the purposes of today, here's what you need to know.

MCMC describes a set of algorithms that define rules to intelligently step in parameter space, with the goal to explore the probability distribution. In the basic concept, starting from some point in parameter space, the algorithm will pick a new point according to some proposal distribution $q(\theta)$. It then compares the posterior probability of the new point with the posterior probability of the old point. If the ratio of the two $r$ is larger than one, the new point is kept. If it's smaller than 1, then the new point will be accepted with probability $r$ and rejected with probability $1-r$. If it's not accepted, the original point will be duplicated, and the procedure starts again. If the new point is accepted, it will become the next point to jump from according to the proposal distribution $q$. In this way, we build up a list of points--or more commonly referred to as *samples*--that is denser in regions of high probability and less dense in regions of low probability. If you run this algorithm for long enough (ideally, to infinity, but who has time for that?), the samples will provide a good approximation to the posterior density. 

The most well-known MCMC algorithm is the [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis–Hastings_algorithm). There are also a number of related methods like Nested Sampling, Slice Sampling and Hamiltonian Monte Carlo, which all approach the sampling problem somewhat differently.  

**Exercise for at home**

Write your own Metropolis-Hastings sampler! There are better algorithms for most problems (though don't underestimate how powerful Metropolis-Hastings can be on the right problem!), but writing your own sampler is invaluable for understanding some of the tricky problems with samplings. I encourage you to write your own, and compare it to samplers available in existing software packages. 

We're not going to write our own sampler here, but rather use an implementation in the popular package `emcee`. If you don't have that package installed, you can do so by typing `pip install emcee` into the empty cell below:

`emcee` works somewhat similar to the Metropolis-Hastings algorithm outlined below, but includes some important improvements. It also doesn't run a single sampling process as we've outlined above, but rather many of them simultaneously. Let's set this up:

In [ ]:
import emcee

In [ ]:
# number of parallel sampling processes
nwalkers = 32

### number of dimensions for the Gaussian seeds (= number of parameters)
ndim = len(res.x)

We need to sample some random starting positions. We're going to do that using the best-fit optimization result above: 

In [ ]:
### sample random starting positions for each of the walkers
p0 = [np.random.multivariate_normal(res.x,res.hess_inv) for i in range(nwalkers)]

In [ ]:
### initialize sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior, args=[x, y, counts])

While we made a good guess that the starting parameters are near the true properties of our source, we don't know that in practice. So initially, it's possible that our sampling processes will be far away from a region of high probability. While MCMC algorithms are designed such that they will (ideally) eventually move to regions of high probability, they might initially not be. This means that the first steps in our process will not accurately reflect the true posterior distribution.

For this reason, we generally throw away the initial $k$ steps in our chain, which is called "burn-in". After that, we hope that the chain is *burned-in*, that is, it has moved to our posterior distribution. Let's try this out:

In [ ]:
burnin_samples = 1000
pos, prob, state = sampler.run_mcmc(p0, burnin_samples)

This might take a few minutes to run, depending on your computer!

One useful check is to plot the all the samples from all the processes. If there are large trends in the samples over time, that is, if the processes wander about, then the processes is likely not burned in yet, and we need to run it for longer. If the chains look like random noise, they might be burned in and good to go.

**Exercise**

Plot the chains for each parameter. Are they stable around some value? Do they wander about in parameter space? 

Hint: You can get the samples using the command `sampler.get_chain()`

Let's run this for longer, and then throw away the first thousand or so samples.

In [ ]:
_,_,_ = sampler.run_mcmc(pos, 1000, progress=True)

In [ ]:
samples = sampler.get_chain()

In [ ]:
samples.shape

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(8,6))

axes = np.hstack(axes)

for i in range(len(res.x)):
    axes[i].plot(samples[:,:,i], alpha=0.1, color="black")

There are other useful ways to check how well your chains are converged to what's hopefully the true posterior distribution (check out the reference at the start of this section for other approaches), but it's worth keeping in mind that truly knowing that this is the case is pretty much impossible in many practical applications. Depending on the structure of the posterior probability, and how and where you initialize your chains, you might entirely miss an important region of high-probability, and never know!

In [ ]:
samples = sampler.get_chain(discard=1000, flat=True)

A useful way to visualize your MCMC chains is using what's called a *corner plot*. This takes your multi-dimensional distribution, and plots each dimension as a histogram, and each pair of dimensions as a contour plot. 

You can write the code to do this yourself (it's not hard), or you can use the package `corner` (install via `pip install corner`) as a convenient visualization tool:

In [ ]:
import corner

In [ ]:
corner.corner(samples);

This posterior looks pretty regular and symmetric, which is nice! You can, of course, summarize this into typical mean and uncertainty estimates:

In [ ]:
parameter_names = ["amplitude", "x coord", "y coord", "width", "bkg"]

for i in range(len(res.x)):
    print("%s posterior: "%parameter_names[i] + str(np.mean(samples[:,i])) + " +/- " + str(np.std(samples[:,i])))

For problems where posteriors are asymmetric or have multiple peaks (also called *modes*), you might prefer the median and quantiles to summarize the distribution, or simply use the corner plot above as your representation of the posterior.

## Exploring (Posterior) Inference

We're not done just having found a maximum likelihood value or a posterior sample. We now have to answer the most important question: **Does our result make sense**? 

This is, of course, a complicated question to answer, and it's very dependent on the problem you're solving. 
But here are some starting questions you can ask yourselves:
* Is the likelihood function I've chosen actually appropriate for the data you have? Or did my data come from a different distribution? Was your collaborator right? Should you use a Gaussian likelihood here? Or do you think you should use a Poisson distribution? Why or why not?
* If I take my best-fit model, or draw samples from the posterior, are the functions it generates good representations of the data I've observed? Hint: This is hard to do in 2D, so I'd recomment plotting the data and model for slices through the image.
* Is my MCMC procedure converged to the posterior?
* Do my priors make sense? Especially hard boundaries on flat priors are a very strong assumption (you're saying there's no way the parameter value could ever lie outside of those boundaries), so checking whether these make sense can be a useful way to troubleshoot.
* Where are there bugs in my code? Because there are always bugs in code. :) 
* What does my posterior look like compared to my prior? Are they close to each other? Are they far away from each other? 

**Useful Information**: If you've done model fitting in the past, you might have heard that the summed squared residuals, or $\chi^2$ divided by the degrees of freedom, should be close to 1 for the model to be a good representation. This works because for data with Gaussian uncertainties (where you use a Gaussian likelihood in your modeling), the sum of the log-likelihood for all data points is distributed following a $\chi^2$ distribution scaled by the degrees of freedom, which in turn means that if you divide by the degrees of freedom, you get a central chi-square distribution with a mean of $1$. The further away you are from that mean, the smaller the probability that your residuals were independent draws from a Gaussian distribution, which they should be if the model were exactly the true, underlying process that generated your observed data. This is a useful diagnostic for checking your model, but it only works well if your likelihood is Gaussian.

**Exercise 1**:

Do you think that your posterior inference gets close to the true parameters? Why or why not? 

**Exercise 2**:

Compare your results with some of the other groups. Are they similar? Are they very different? If they are different, can you figure out why? 